In [ ]:
# !pip install comfit -q
import sys
from pathlib import Path
current_dir = Path().resolve()
parent_dir = current_dir.parent
sys.path.append(str(parent_dir))
import comfit as cf
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy as sp


If you find any errors in this tutorial, please raise a issue at [GitHub](https://github.com/vidarsko/ComFiT/issues).

# Phase-field crystal tutorial: Stresses and strains

In this tutorial, we will have a look at stresses and strains in the phase-field crystal. 

We start by initializing a phase-field crystal with a dislocation dipole, if you are confused by this procedure, check out the [PFC basic framework tutorial](https://colab.research.google.com/github/vidarsko/ComFiT/blob/main/tutorial/phase_field_crystal_basic_framework.ipynb). 

In [ ]:
import comfit as cf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
pfc = cf.PhaseFieldCrystal2DTriangular(30, 15)
eta = pfc.calc_amplitudes_with_dislocation_dipole()
pfc.conf_PFC_from_amplitudes(eta)
pfc.evolve_PFC(100)
pfc.plot_field(pfc.psi)

Now we calculate the stress field 

In [ ]:
stress =  pfc.calc_stress_tensor()
pfc.plot_field(stress[0])

Above, you see the $xx$ component of the stress, the circular region is an artifact of the initial condition (see [documentation](https://comfitlib.com/ClassBaseSystem/#periodic-boundary-conditions-numerical-implementation-of-angle-fields)), and will disappear as the simulation progresses.


The stress tensor is a symmetric tensor and therefore only has three independent components. 
To save memory, these are saved in an array structure `stress[0]`, `stress[1]`, and `stress[2]`, where `stress[0]` is the $xx$ component, `stress[1]` is the $xy$ component, and `stress[2]` is the $yy$ component.
You can extract a specific component from this tensor by using the `get_sym` method.

In [ ]:
sigmaxx = pfc.get_sym(stress,0,0)
sigmaxy = pfc.get_sym(stress,0,1)
sigmayx = pfc.get_sym(stress,1,0)
sigmayy = pfc.get_sym(stress,1,1)

For more information on this point, see [documentation](https://comfitlib.com/ClassBaseSystem/). 

Let us plot all the components in units of the elastic shear modulus `el_mu`. 

In [ ]:
# Make empty list to store figures
figs = []

for i in range(2):
    for j in range(2):
        # Add figure to list
        figs.append(pfc.plot_field(pfc.get_sym(stress,i,j)/pfc.el_mu))

# Create subplots, first two arguments specify the number of rows and columns
cf.tool_make_subplots(2,2,*figs)

# Alternatively: cf.tool_make_subplots(2,2,fig[0],fig[1],fig[2],fig[3])

Now, lets have a look at stresses for dislocations in other symmetries.
For information on how these are calculated, see the [documentation](https://comfitlib.com/ClassPhaseFieldCrystal/#elasticity). 

In [ ]:
pfc = cf.PhaseFieldCrystal2DSquare(30, 30)
eta = pfc.calc_amplitudes_with_dislocation_dipole(dislocation_type=2)
pfc.conf_PFC_from_amplitudes(eta)
pfc.evolve_PFC(100)

stress = pfc.calc_stress_tensor()

figs = []
for i in range(2):
    for j in range(2):
        figs.append(pfc.plot_field(pfc.get_sym(stress,i,j)/pfc.el_mu))

cf.tool_make_subplots(2,2,*figs)

In [ ]:
pfc = cf.PhaseFieldCrystal3DBodyCenteredCubic(13, 13, 13)
eta = pfc.calc_amplitudes_with_dislocation_ring()
pfc.conf_PFC_from_amplitudes(eta)
pfc.evolve_PFC(10)

stress = pfc.calc_stress_tensor()

figs = []

for i in range(6):
    figs.append(pfc.plot_field(stress[i]/pfc.el_mu))

cf.tool_make_subplots(2,3,*figs)

This might not be so informative, so it might be useful to see the plot of the stress field for the xx-component in a plane.

In [ ]:
pfc.plot_field_in_plane(stress[0],normal_vector=[0,1,0])

In [ ]:
# pfc = cf.PhaseFieldCrystal3DFaceCenteredCubic(9, 9, 9)
# eta = pfc.calc_amplitudes_with_dislocation_ring()
# pfc.conf_PFC_from_amplitudes(eta)
# pfc.evolve_PFC(10)

# stress = pfc.calc_stress_tensor()

# pfc.plot_field_in_plane(stress[1]/pfc.el_mu)

In [ ]:
pfc = cf.PhaseFieldCrystal3DSimpleCubic(13, 14, 15)
eta = pfc.calc_amplitudes_with_dislocation_ring()
pfc.conf_PFC_from_amplitudes(eta)
pfc.evolve_PFC(10)

stress = pfc.calc_stress_tensor()

pfc.plot_field_in_plane(stress[3]/pfc.el_mu)


We may also have a look at the strain field of the PFC by calculating the strain using the method described in the [documentation](https://comfitlib.com/ClassPhaseFieldCrystal/).

In [ ]:
pfc = cf.PhaseFieldCrystal2DTriangular(40, 28)
eta = pfc.calc_amplitudes_with_dislocation_dipole()
pfc.conf_PFC_from_amplitudes(eta)

pfc.evolve_PFC(100)

strain = pfc.calc_strain_tensor()
pfc.plot_field(strain[0])


This shows the $xx$ component of the strain. 
There is some residual stress in the system though, due to the periodic arrangement of peaks and troughs not exactly matching the periodic boundary conditions, see [documentation](https://comfitlib.com/ClassPhaseFieldCrystal/#elasticity). 
For visualization purposes then, it is useful to subtract the mean value, which we can do by


In [ ]:
strain = strain - np.mean(strain,axis=0) 
pfc.plot_field(strain[2])